## Queries correlacionados

- Son utilizados cuando se quiere tener un resultado distinto para cada fila considerada en el query principal
<br>.
- Es un subquery que hace referencia a una columna de una tabla que no está en el FROM
<br>.
- Se dice que el query está correlacionado porque el resultado que produce depende de un valor que produce el SELECT externo, es por ello que el subquery se debe ejecutar una vez por cada fila que considera el query externo
<br>.
- No se pueden ejecutar sin el query principal
<br>.
- Son ineficientes en cuanto a tiempo de ejecución

In [ ]:
Estructura de query correlacionado:
    
'''
SELECT column1, column2, ....
FROM table1 outer
WHERE column1 operator
                    (SELECT column1, column2
                     FROM table2
                     WHERE expr1 = 
                               outer.expr2);
                        '''

## Funcionamiento de query correlacionado

### GET
candidate row from outer query

### EXECUTE
inner query using candidate row value

### USE
values from inner query to qualify or disqualify candidate row

In [4]:
pedidos = spark.sql('''
select folio_pedido, persona, productos
from base_pedidos
'''
)
pedidos.persist()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[folio_pedido: bigint, persona: bigint, productos: bigint]

In [5]:
pedidos.createOrReplaceTempView('pedidos')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Ejemplo
Se cuenta con la base de datos de una tienda de ventas en línea, donde se tiene el registro de los pedidos, id de cliente que realiza el pedido y el número de productos que se ordenan por pedido.
Se quiere encontrar los pedidos que tienen más productos comprados que el promedio por pedido de por persona.

In [6]:
query = spark.sql('''
SELECT folio_pedido, persona, productos
FROM pedidos outer
WHERE productos >
                (SELECT AVG(productos)
                 FROM pedidos
                 WHERE persona =
                        outer.persona)
ORDER BY persona

''')
query.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------+---------+
|folio_pedido| persona|productos|
+------------+--------+---------+
|   558064744|11013560|        3|
|   552388905|11013560|        3|
|   697217172|13302208|        4|
|   690768520|13302208|        4|
|   635984960|18066926|        2|
|   689627701|19648519|        2|
|   657096359|19720128|        3|
|   526331055|20802055|       11|
|   521483505|20984688|        2|
|   562674850|20984688|        2|
|   591594773|21702225|        2|
|   594783274|21702225|        2|
|   592125024|21702225|        2|
|   718087557|22231809|        3|
|   548600130|22323644|       14|
|   523462228|22917087|        5|
|   492256010|24735617|        8|
|   644172615|26123713|        4|
|   575041454|26349968|        4|
|   623183509|28427666|        3|
+------------+--------+---------+
only showing top 20 rows

Pedidos de la persona 11013560

In [8]:
df = spark.sql('''
SELECT folio_pedido, persona, productos
FROM pedidos
where persona = 11013560
''')
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+--------+---------+
|folio_pedido| persona|productos|
+------------+--------+---------+
|   558181484|11013560|        2|
|   558064744|11013560|        3|
|   552388905|11013560|        3|
+------------+--------+---------+

Promedio de productos comprados por pedido de la persona 11013560

In [10]:
df = spark.sql('''
SELECT AVG(productos)
FROM pedidos
where persona = 11013560
''')
df.show()


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|    avg(productos)|
+------------------+
|2.6666666666666665|
+------------------+